## Short Straddle Calculator

In [3]:
import QuantLib as ql
import pandas as pd

In [4]:
def short_straddle_calc(initiation, DTE, strike, spot, IV, divd, rf, market='NullCalendar'):
    """
    Calculates the Short Straddle value and greeks
    initiation: date as serialNumber (excel format), eg. 42310 = 2025-11-02
    IV (implied volatility), divd (dividends) and rf (riskfree rate) as decimal
    market: the country calender, defaults to NullCalendar
    """

    trade_date = ql.Date(initiation)
    ql.Settings.instance().evaluationDate = trade_date
    try:
        calendar = getattr(ql, market)()
    except AttributeError:
        raise ValueError(f"Unknown market calendar: {market}")

    expiry = ql.Date(initiation) + DTE
    strike_price = strike
    call = ql.Option.Call
    put = ql.Option.Put
    spot_price = spot
    volatility = IV
    dividend_rate = divd
    risk_free_rate = rf

    call_payoff = ql.PlainVanillaPayoff(call, strike_price)
    put_payoff = ql.PlainVanillaPayoff(put, strike_price)
    
    exercise = ql.EuropeanExercise(expiry)
    call_option = ql.VanillaOption(call_payoff, exercise)
    put_option = ql.VanillaOption(put_payoff, exercise)

    spot_handle = ql.QuoteHandle(ql.SimpleQuote(spot_price))
    volatility_handle = ql.BlackVolTermStructureHandle(
        ql.BlackConstantVol(ql.Date(initiation), calendar, ql.QuoteHandle(ql.SimpleQuote(volatility)), ql.Actual365Fixed()))
    dividend_handle = ql.YieldTermStructureHandle(
        ql.FlatForward(ql.Date(initiation), ql.QuoteHandle(ql.SimpleQuote(dividend_rate)), ql.Actual365Fixed()))
    risk_free_handle = ql.YieldTermStructureHandle(
        ql.FlatForward(ql.Date(initiation), ql.QuoteHandle(ql.SimpleQuote(risk_free_rate)), ql.Actual365Fixed()))

    bsm_process = ql.BlackScholesMertonProcess(spot_handle, dividend_handle, risk_free_handle, volatility_handle)

    call_option.setPricingEngine(ql.AnalyticEuropeanEngine(bsm_process))
    put_option.setPricingEngine(ql.AnalyticEuropeanEngine(bsm_process))

    call_price = call_option.NPV()
    put_price = put_option.NPV()
    call_delta = call_option.delta()
    put_delta = put_option.delta()
    call_gamma = call_option.gamma()
    put_gamma = put_option.gamma()
    call_vega = call_option.vega()
    put_vega = put_option.vega()
    call_theta = call_option.theta()
    put_theta = put_option.theta()
    
    straddle_value = call_price + put_price
    straddle_delta = -(call_delta + put_delta)
    straddle_gamma = -(call_gamma + put_gamma)
    straddle_vega = -(call_vega + put_vega) / 100
    straddle_theta = -(call_theta + put_theta) / 100

    return pd.Series(
        {"Value": straddle_value,
         "Delta": straddle_delta,
         "Gamma": straddle_gamma,
         "Vega":  straddle_vega,
         "Theta": straddle_theta
        })

In [5]:
short_straddle_calc(initiation=42310, DTE=21, strike=1502.437012, spot=1502.437012, IV=0.19854101, divd=0, rf=0.00674, market='Sweden')

Value    57.074082
Delta    -0.025491
Gamma    -0.011146
Vega     -2.873941
Theta     4.957478
dtype: float64